In [44]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import WazeRouteCalculator
import datetime

## Pulling Job Data

In [45]:
#project_path = "/Users/jjespinoza/GoogleDrive/2_projects/2_itsfriday/"
project_path = "/home/jj_espinoza_la/itsfriday/"

now = datetime.datetime.now()
now.year
now.month
now.day
date = str(now.year) + str(now.month) + str(now.day) + "_" + str(now.hour) + str(now.minute)
date


'2018117_1612'

In [46]:
max_results_per_city = 20
city_set = ['Los+Angeles']
# city_set = ['Los+Angeles', 'Los+Angeles+County', 'Long+Beach', 'Downey', 'Commerce']
title_set = ['data+scientist']
#title_set = ['data+scientist', 'senior+data+scientist', 'director+data+science', 'director+analytics', 'vice+president+analytics', 'vice+president+data+science']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']
sample_df = pd.DataFrame(columns = columns)


In [47]:
#scraping code:
for title in title_set:
    for city in city_set:
        for start in range(0, max_results_per_city, 10):
            page = requests.get('http://www.indeed.com/jobs?q='+ str(title) +'+%2420%2C000&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)  #ensuring at least 1 second between page grabs
            soup = BeautifulSoup(page.text, 'lxml')
            for div in soup.find_all(name='div', attrs={'class':'row'}): 
                #specifying row num for index of job posting in dataframe
                num = (len(sample_df) + 1) 
                #creating an empty list to hold the data for each posting
                job_post = [] 
                #append city name
                job_post.append(city) 
                #grabbing job title
                for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                    job_post.append(a['title']) 
                #grabbing company name
                company = div.find_all(name='span', attrs={'class':'company'}) 
                if len(company) > 0: 
                    for b in company:
                        job_post.append(b.text.strip()) 
                else: 
                    sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                    for span in sec_try:
                        job_post.append(span.text) 
                #grabbing location name
                c = div.findAll('span', attrs={'class': 'location'}) 
                for span in c: 
                    job_post.append(span.text) 
                #grabbing summary text
                d = div.findAll('span', attrs={'class': 'summary'}) 
                for span in d:
                    job_post.append(span.text.strip()) 
                #grabbing salary
                try:
                    job_post.append(div.find('nobr').text) 
                except:
                    try:
                        div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                        div_three = div_two.find('div') 
                        job_post.append(div_three.text.strip())
                    except:
                        job_post.append('Nothing_found') 
                #appending list of job post info to dataframe at index num
                sample_df.loc[num] = job_post




In [48]:
del sample_df['city']

In [49]:
sample_df = sample_df.drop_duplicates()

In [50]:
sample_df

,job_title,company_name,location,summary,salary
1,Senior Data Scientist,Restoration Media,"Irvine, CA 92614",The Senior Data Scientist will leverage the bu...,Nothing_found
2,Senior Data Scientist,LT,"Los Angeles, CA",The ideal candidate for this Senior Data Scien...,"$150,000 - $190,000 a year"
3,Director of Data & Analytics,Joybird,"Commerce, CA",3+ years experience in an analytical role or a...,Nothing_found
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found
6,Data Scientist / Data Analyst,Hyperloop One,"Los Angeles, CA",The Data Scientist / Data Analyst will be part...,Nothing_found
7,"Research Scientist, Machine Learning / Artific...",Disney Parks & Resorts,"Glendale, CA",Duties of a Research Scientist include:. Resea...,Nothing_found
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found
10,Associate Machine Learning Engineer,eharmony,"Los Angeles, CA",Create data pipelines to establish datasets fo...,Nothing_found


## Dropping unwanted job titles


In [51]:

sample_df = sample_df[(sample_df["job_title"] != "Account Director") & 
                       (~sample_df["job_title"].str.contains('Software Engineer')) &
                      (~sample_df["job_title"].str.contains('Intern')) &
                      (~sample_df["job_title"].str.contains('Engineer')) &
                      (~sample_df["job_title"].str.contains('Research')) &
                     (~sample_df["job_title"].str.contains('Manager')) & 
                     (~sample_df["job_title"].str.contains('Analyst'))]

sample_df

,job_title,company_name,location,summary,salary
1,Senior Data Scientist,Restoration Media,"Irvine, CA 92614",The Senior Data Scientist will leverage the bu...,Nothing_found
2,Senior Data Scientist,LT,"Los Angeles, CA",The ideal candidate for this Senior Data Scien...,"$150,000 - $190,000 a year"
3,Director of Data & Analytics,Joybird,"Commerce, CA",3+ years experience in an analytical role or a...,Nothing_found
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found
13,"Associate, Data Scientist",KPMG,"Los Angeles, CA 90071","Retrieve, process and prepare a rich data vari...",Nothing_found
14,Seeking full time Clinical Labratory Scientist,Epicgenetics,"Los Angeles, CA 90067",Performs clinical laboratory assays to provide...,Nothing_found
15,Principal Data Scientist,sartorius,California,Sartorius - Principal Engineer/Principal Data ...,Nothing_found


## Dropping Unwanted Cities

In [52]:
sample_df = sample_df[ (~sample_df["location"].str.contains('Woodland Hills')) &
                      (~sample_df["location"].str.contains('Santa Monica')) &
                      (~sample_df["location"].str.contains('Culver City')) &
                      (~sample_df["location"].str.contains('Burbank')) &
                      (~sample_df["location"].str.contains('Hollywood')) &
                      (~sample_df["location"].str.contains('Irvine')) &
                      (~sample_df["location"].str.contains('Venice'))
                     ]

sample_df

,job_title,company_name,location,summary,salary
2,Senior Data Scientist,LT,"Los Angeles, CA",The ideal candidate for this Senior Data Scien...,"$150,000 - $190,000 a year"
3,Director of Data & Analytics,Joybird,"Commerce, CA",3+ years experience in an analytical role or a...,Nothing_found
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found
13,"Associate, Data Scientist",KPMG,"Los Angeles, CA 90071","Retrieve, process and prepare a rich data vari...",Nothing_found
14,Seeking full time Clinical Labratory Scientist,Epicgenetics,"Los Angeles, CA 90067",Performs clinical laboratory assays to provide...,Nothing_found
15,Principal Data Scientist,sartorius,California,Sartorius - Principal Engineer/Principal Data ...,Nothing_found
20,Fraud Data Scientist,Stamps.com,"El Segundo, CA 90245",Analyze large data sets to identify behavior t...,Nothing_found


## Dropping Companies

In [53]:
sample_df = sample_df[ (~sample_df["company_name"].str.contains('UCLA Extension')) &
                      (~sample_df["company_name"].str.contains('USC')) &
                      (~sample_df["company_name"].str.contains('Southern California University of Health Sciences')) &
                      (~sample_df["company_name"].str.contains('Riot Games')) &
                      (~sample_df["company_name"].str.contains('Los Angeles County')) &
                      (~sample_df["company_name"].str.contains('Lieberman Research Worldwide')) &
                      (~sample_df["company_name"].str.contains('Childrens Hospital Los Angeles')) &
                      (~sample_df["company_name"].str.contains('CEDARS-SINAI')) &
                      (~sample_df["company_name"].str.contains('California State University')) &
                      (~sample_df["company_name"].str.contains('First 5 LA'))
                     ]





sample_df

,job_title,company_name,location,summary,salary
2,Senior Data Scientist,LT,"Los Angeles, CA",The ideal candidate for this Senior Data Scien...,"$150,000 - $190,000 a year"
3,Director of Data & Analytics,Joybird,"Commerce, CA",3+ years experience in an analytical role or a...,Nothing_found
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found
13,"Associate, Data Scientist",KPMG,"Los Angeles, CA 90071","Retrieve, process and prepare a rich data vari...",Nothing_found
14,Seeking full time Clinical Labratory Scientist,Epicgenetics,"Los Angeles, CA 90067",Performs clinical laboratory assays to provide...,Nothing_found
15,Principal Data Scientist,sartorius,California,Sartorius - Principal Engineer/Principal Data ...,Nothing_found
20,Fraud Data Scientist,Stamps.com,"El Segundo, CA 90245",Analyze large data sets to identify behavior t...,Nothing_found


## Calculating Distance

In [54]:
sample_df["clean_address"] = sample_df["company_name"] + ' ' + sample_df["location"]

In [55]:
commutes = []

for address in sample_df["clean_address"]:
    from_address = "10757 Longworth Ave Santa Fe Springs CA 90670"
    region = 'US'
    try:
        to_address = address
        route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region )
        commutes.append(route.calc_route_info(real_time=False))
    except:
        commutes.append((0,0))
    
    
        
    
        

From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: LT Los Angeles, CA
Time 70.58 minutes, distance 45.39 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Joybird Commerce, CA
Time 13.27 minutes, distance 10.87 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: KPMG Los Angeles, CA 90071
Time 70.58 minutes, distance 45.39 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Twentieth Century Fox Los Angeles, CA
Time 71.77 minutes, distance 67.54 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Next Trucking Los Angeles, CA
Time 70.53 minutes, distance 45.39 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Internet Brands Los Angeles, CA
Time 70.30 minutes, distance 63.47 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: KPMG Los Angeles, CA 90071
Time 70.53 minutes, distance 45.39 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: Epicgenetics Los Angeles, CA 90067
Time 70.87 minutes, distance 45.39

In [56]:
df = pd.DataFrame(commutes) 
df.columns = ['commute', 'distance_km']
df.index = range(1,len(df)+1)
sample_df = sample_df.join(df, lsuffix='_caller', rsuffix='_other')
sample_df


,job_title,company_name,location,summary,salary,clean_address,commute,distance_km
2,Senior Data Scientist,LT,"Los Angeles, CA",The ideal candidate for this Senior Data Scien...,"$150,000 - $190,000 a year","LT Los Angeles, CA",13.266667,10.871
3,Director of Data & Analytics,Joybird,"Commerce, CA",3+ years experience in an analytical role or a...,Nothing_found,"Joybird Commerce, CA",70.583333,45.394
4,"Associate, Data Scientist/Statistics - Financi...",KPMG,"Los Angeles, CA 90071","Machine learning, data visualization, statisti...",Nothing_found,"KPMG Los Angeles, CA 90071",71.766667,67.542
5,Artificial Intelligence & Data Scientist,Twentieth Century Fox,"Los Angeles, CA",1-3+ years of experience as data scientist. Ex...,Nothing_found,"Twentieth Century Fox Los Angeles, CA",70.533333,45.394
8,Data Scientist - Tech Start Up,Next Trucking,"Los Angeles, CA",We are looking for a motivated Data Scientist ...,Nothing_found,"Next Trucking Los Angeles, CA",70.866667,45.394
9,Data Scientist – Machine Learning,Internet Brands,"Los Angeles, CA",Is currently seeking a Data Scientist - Machin...,Nothing_found,"Internet Brands Los Angeles, CA",356.833333,630.465
13,"Associate, Data Scientist",KPMG,"Los Angeles, CA 90071","Retrieve, process and prepare a rich data vari...",Nothing_found,"KPMG Los Angeles, CA 90071",NaN,NaN
14,Seeking full time Clinical Labratory Scientist,Epicgenetics,"Los Angeles, CA 90067",Performs clinical laboratory assays to provide...,Nothing_found,"Epicgenetics Los Angeles, CA 90067",NaN,NaN
15,Principal Data Scientist,sartorius,California,Sartorius - Principal Engineer/Principal Data ...,Nothing_found,sartorius California,NaN,NaN
20,Fraud Data Scientist,Stamps.com,"El Segundo, CA 90245",Analyze large data sets to identify behavior t...,Nothing_found,"Stamps.com El Segundo, CA 90245",NaN,NaN


In [57]:
sample_df.to_csv(project_path + "2_data/1_raw/scraper_indeed_jobs_" + date + ".csv", encoding='utf-8')